<a href="https://colab.research.google.com/github/AleAguirreM/LEA3_Health/blob/main/c_modelos_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import joblib ### para cargar array

########Paquetes para NN #########
import tensorflow as tf
from sklearn import metrics ### para analizar modelo
from sklearn.ensemble import RandomForestClassifier  ### para analizar modelo
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import keras

In [3]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [4]:
### cargar bases_procesadas ####

x_train = joblib.load('/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/x_train.pkl')
y_train = joblib.load('/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/y_train.pkl')
x_test = joblib.load('/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/x_test.pkl')
y_test = joblib.load('/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/y_test.pkl')


In [5]:
#### Escalar ######################
x_train=x_train.astype('float32') ## para poder escalarlo
x_test=x_test.astype('float32') ## para poder escalarlo
x_train /=255 ### escalaro para que quede entre 0 y 1
x_test /=255

In [6]:
###### verificar tamaños

print(x_train.shape)
x_test.shape


(3360, 100, 100, 3)


(840, 100, 100, 3)

In [7]:
np.product(x_train[1].shape) ## cantidad de variables por imagen

30000

In [8]:
print(np.unique(y_train, return_counts=True))
np.unique(y_test, return_counts=True)

(array([0, 1]), array([2800,  560]))


(array([0, 1]), array([700, 140]))

In [25]:
##########################################################
################ Redes convolucionales ###################
##########################################################

cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[1:]),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with binary cross-entropy loss and Adam optimizer
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC', 'Precision','Accuracy'])

In [26]:
# Train the model for 10 epochs
cnn_model.fit(x_train, y_train, batch_size=100, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
34/34 [==============================] - 32s 903ms/step - loss: 0.3396 - auc: 0.8142 - precision: 0.7126 - Accuracy: 0.8646 - val_loss: 0.3446 - val_auc: 0.9235 - val_precision: 0.6667 - val_Accuracy: 0.8357
Epoch 2/10
34/34 [==============================] - 28s 831ms/step - loss: 0.1755 - auc: 0.9532 - precision: 0.8801 - Accuracy: 0.9333 - val_loss: 0.3447 - val_auc: 0.9594 - val_precision: 0.9000 - val_Accuracy: 0.8429
Epoch 3/10
34/34 [==============================] - 30s 892ms/step - loss: 0.1214 - auc: 0.9827 - precision: 0.9189 - Accuracy: 0.9533 - val_loss: 0.2011 - val_auc: 0.9743 - val_precision: 0.9455 - val_Accuracy: 0.8917
Epoch 4/10
34/34 [==============================] - 27s 785ms/step - loss: 0.1102 - auc: 0.9834 - precision: 0.9374 - Accuracy: 0.9580 - val_loss: 0.1728 - val_auc: 0.9788 - val_precision: 0.9398 - val_Accuracy: 0.9202
Epoch 5/10
34/34 [==============================] - 30s 905ms/step - loss: 0.0836 - auc: 0.9903 - precision: 0.9679 - Accura

Analizando la métrica AUC se observan buenos rendimientos, ya que tanto en entrenamiento como en evaluación tiene valores muy buenos de aproximadamente 0,99, por lo que no se han identificado problemas de subajuste (underfitting) ni de sobreajuste (overfitting), lo que sugiere que no será necesario implementar técnicas de regularización ni afinamiento de hiperparametros en este momento.

In [27]:
#################### Mejor redes ##############

joblib.dump(cnn_model, "/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/cnn_model.pkl")

['/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/cnn_model.pkl']